In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#define path
path = r'C:\Users\rtkel\Documents\Data Analytics Course\Instacart Basket Analysis'

In [3]:
path

'C:\\Users\\rtkel\\Documents\\Data Analytics Course\\Instacart Basket Analysis'

In [4]:
#Import ords_prods_merged_updated
ords_prods_merged_updated = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_updated.pkl'))

In [6]:
#shape
ords_prods_merged_updated.shape

(32434212, 19)

In [7]:
#head
ords_prods_merged_updated.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,hour_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,product_name,aisle_id,department_id,prices,_merge,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,prior,1,2,8,NaN,196,1,0,195,Soda,77,7,9.0,both,Regularly busy,Regularly days,Average orders
1,2398795,1,prior,2,3,7,15.0,196,1,1,195,Soda,77,7,9.0,both,Regularly busy,Slow days,Average orders
2,473747,1,prior,3,3,12,21.0,196,1,1,195,Soda,77,7,9.0,both,Regularly busy,Slow days,Most orders
3,2254736,1,prior,4,4,7,29.0,196,1,1,195,Soda,77,7,9.0,both,Least busy,Slow days,Average orders
4,431534,1,prior,5,4,15,28.0,196,1,1,195,Soda,77,7,9.0,both,Least busy,Slow days,Most orders


#### 2. In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [8]:
ords_prods_merged_updated.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.213779
6,16.439806
7,17.225773
8,15.340520


#### 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

The averages of the subset seem to be higher than the entire dataframe's averages as a general trend.

#### 4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [9]:
#Generate max orders column
ords_prods_merged_updated['max_order'] = ords_prods_merged_updated.groupby(['user_id'])['order_number'].transform(np.max)

In [10]:
#Create loyalty flag
ords_prods_merged_updated.loc[ords_prods_merged_updated['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [11]:
ords_prods_merged_updated.loc[(ords_prods_merged_updated['max_order'] <= 40) & (ords_prods_merged_updated['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [12]:
ords_prods_merged_updated.loc[ords_prods_merged_updated['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
#Check frequency
ords_prods_merged_updated['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: count, dtype: int64

#### 5. The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [14]:
ords_prods_merged_updated.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.388747  1.0  99999.0
New customer      13.294370  1.0  99999.0
Regular customer  12.496203  1.0  99999.0

#### 6. The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [15]:
#Generate mean product price for each user
ords_prods_merged_updated['mean_price'] = ords_prods_merged_updated.groupby(['user_id'])['prices'].transform(np.mean)

In [16]:
#create flags
ords_prods_merged_updated.loc[ords_prods_merged_updated['mean_price'] < 10, 'spending_flag'] = 'Low spender'

In [17]:
ords_prods_merged_updated.loc[ords_prods_merged_updated['mean_price'] >= 10, 'spending_flag'] = 'High spender'

In [18]:
#Check frequencies
ords_prods_merged_updated['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31798751
High spender      635461
Name: count, dtype: int64

#### 7. In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [19]:
#Genrate median of days since prior order by user id
ords_prods_merged_updated['median_days'] = ords_prods_merged_updated.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
#Creat flags
ords_prods_merged_updated.loc[ords_prods_merged_updated['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [21]:
ords_prods_merged_updated.loc[(ords_prods_merged_updated['median_days'] > 10) & (ords_prods_merged_updated['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'

In [22]:
ords_prods_merged_updated.loc[ords_prods_merged_updated['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [23]:
#Check frequencies
ords_prods_merged_updated['frequency_flag'].value_counts(dropna=False)

frequency_flag
Frequent customer        21577409
Regular customer          7217134
Non-frequent customer     3639669
Name: count, dtype: int64

In [24]:
ords_prods_merged_updated.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merged_updated_2.pkl'))